In [2]:
# imports

from functools import cache
import json
from collections import Counter, defaultdict
from itertools import permutations, combinations
import plotly.express as px

import string
import pandas as pd

from diff_functions import diff2_inline, diff_inline, diff_n, score_n_all, score_all


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
# load words

def load_words(filename):
    with open(filename) as f:
        return f.read().split()

core = load_words('core.txt')
extended = load_words('extended.txt')
print(len(core), len(extended))


2315 12953


In [13]:
g, y = diff_inline("abcde", "abcde")
import sys

print(sys.getsizeof(g), sys.getsizeof(y))
print(sys.getsizeof(tuple(g)), sys.getsizeof(tuple(y)))

120 56
80 40


In [ ]:
# stats

import math

def most_common(corpus, n=15):
    freq = Counter()
    for word in corpus:
        freq.update(word)
    df = pd.DataFrame(list(zip(*freq.most_common(26)))).T
    df.columns = ['Letter', 'Count']
    df['Count'] = df['Count'].astype("int32")
    total = sum(df['Count'])
    df['Frequency'] = df['Count'] / total
    print(df)
    fig = px.bar(df, x='Letter', y='Frequency', color='Frequency', color_continuous_scale="viridis")
    fig.update_coloraxes(showscale=False)
    fig.show()
    return [e[0] for e in freq.most_common(n)]

print(most_common(core))
# print(most_common(extended))



In [ ]:
# More stats 

def histogram(corpus):
    counts = [[0,0,0,0,0] for i in range(26)]
    offset = ord('A')
    for word in corpus:
        for idx, letter in enumerate(word):
            counts[ord(letter) - offset][idx] += 1
    return counts

counts = histogram(core)

counts_df = pd.DataFrame(counts)
counts_df.columns = [str(i) for i in range(1, 6)]
counts_df['Letter'] = [chr(i + ord('A')) for i in range(26)]
sorted_last = counts_df.sort_values(by='5', ascending=False)
sorted_last = sorted_last[['Letter', '5']]
total = sum(sorted_last['5'])
label = 'Frequency of last word letter'
sorted_last[label] = sorted_last['5'] / total
# print(sorted_last)
last_freq = px.bar(sorted_last, x='Letter', y=label, color=label, color_continuous_scale="viridis")
last_freq.update_coloraxes(showscale=False)
last_freq.show()

In [ ]:
# entropy

def entropy(letter_counts):
    total = sum(letter_counts)
    probs = [ x / total for x in letter_counts ]
    return -sum([ p * math.log(p, 2) for p in probs if p > 0])

for i in range(5):
    print(i, entropy([c[i] for c in counts]))

global_freqs = [sum(c) for c in counts]
print(entropy(global_freqs))
total = sum(global_freqs)
probs = [ x / total for x in global_freqs ]
for i in range(26):
    print(chr(i + ord('A')), -math.log(probs[i], 2))

In [17]:
# compute match between two words
WELL_PLACED_BONUS = 2

def score(word1, word2):
    well_placed = sum([1 if c1 == c2 else 0 for c1, c2 in zip(word1, word2)])
    letters1 = Counter(word1)
    letters2 = Counter(word2)
    matches = 0
    for letter, count in letters1.items():
        matches += min(count, letters2.get(letter, 0))
    return (well_placed, matches - well_placed)

def score_inline(word1, word2):
    wp, np = diff_inline(word1, word2)
    wpcount = sum([1 if a else 0 for a in wp])
    return (wpcount, len(np))

assert score("aaaaa", "bbbbb") == (0,0)
assert score("abcde", "axxxx") == (1,0)
assert score("abcde", "abcde") == (5,0)
assert score("aabcd", "aabcd") == (5,0)
assert score("abcde", "bcdea") == (0,5)
assert score("aayyy", "axxxa") == (1,1)
assert score("axaxc", "xxaab") == (2,2)

assert score_inline("aaaaa", "bbbbb") == (0,0)
assert score_inline("abcde", "axxxx") == (1,0)
assert score_inline("abcde", "abcde") == (5,0)
assert score_inline("aabcd", "aabcd") == (5,0)
assert score_inline("abcde", "bcdea") == (0,5)
assert score_inline("aayyy", "axxxa") == (1,1)
assert score_inline("axaxc", "xxaab") == (2,2)

def diff(word1, word2):
    well_placed = [c1 if c1 == c2 else None for c1, c2 in zip(word1, word2)]
    well_placed_counter = Counter(well_placed)
    letters1 = Counter(word1)
    letters2 = Counter(word2)
    not_placed = []
    for letter, count in letters1.items():
        matches = min(count, letters2.get(letter, 0)) - well_placed_counter.get(letter, 0)
        not_placed.extend([letter]*matches)
    return well_placed, not_placed

assert diff("aaaaa", "bbbbb") == ([None]*5,[])
assert diff("abcde", "axxxx") == (['a',None,None,None,None],[])
assert diff("abcde", "abcde") == (['a','b','c','d','e'],[])
assert diff("aabcd", "aabcd") == (['a','a','b','c','d'],[])
assert diff("abcde", "bcdea") == ([None]*5,['a','b','c','d','e'])
assert diff("aayyy", "axxxa") == (['a',None,None,None,None],['a'])
assert diff("axaxc", "xxaab") == ([None,'x','a',None,None],['a','x'])

def delta(well_placed, not_placed, well_placed2):
    for (a,b) in zip(well_placed, well_placed2):
        #if letter is well placed in first word AND not well placed in second word
        if a and not b and a in not_placed:
            not_placed.remove(a)
    return not_placed

def merge(list1, list2):
    result = []
    for a in list1:
        if a in list2:
            list2.remove(a)
        result.append(a)
    result.extend(list2)
    return result

def diff2(base, word1, word2):
    well_placed1, not_placed1 = diff(base, word1)
    well_placed2, not_placed2 = diff(base, word2)
    well_placed = [a or b for (a,b) in zip(well_placed1, well_placed2)]
    not_placed = merge(delta(well_placed1, not_placed2, well_placed2), delta(well_placed2, not_placed1, well_placed1))
    return well_placed, not_placed

def compare(diff_result, str1, str2):
    wp, np = diff_result
    wp2 = [None if c == '*' else c for c in str1]
    np2 = list(str2)
    assert wp == wp2
    assert(sorted(np) == sorted(np2))

def score2_generic(diff2_func, base, w1, w2):
    wp, np = diff2_func(base, w1, w2)
    wpcount = sum([1 if a else 0 for a in wp])
    return wpcount*WELL_PLACED_BONUS + len(np)


compare(diff2("aaaaa", "bbbbb", "ccccc"), '*****', '')
compare(diff2("aaaaa", "abbbb", "aaccc"), 'aa***', '')
compare(diff2("aaxxx", "abbbb", "acxca"), 'a*x**', 'a')
compare(diff2("aaxxx", "bbbaa", "cxxca"), '**x**', 'aax')
compare(diff2("toxic","train","taper"), 't**i*', '')
compare(diff2("rapid","train","taper"), '*api*','r')
compare(diff2("affix","taper","bring"), '*****','ai')
compare(diff2("tapir","train","taper"), 'tapir', '')
compare(diff2("tapir","train","paste"), 'ta*i*', 'rp')

assert diff("aaaaa", "bbbbb") == diff2("aaaaa", "bbbbb", "bbbbb")
assert diff("abcde", "axxxx") == diff2("abcde", "axxxx", "axxxx")
assert diff("abcde", "abcde") == diff2("abcde", "abcde", "abcde")
assert diff("aabcd", "aabcd") == diff2("aabcd", "aabcd", "aabcd")
assert diff("abcde", "bcdea") == diff2("abcde", "bcdea", "bcdea")
assert diff("aayyy", "axxxa") == diff2("aayyy", "axxxa", "axxxa")
assert diff("axaxc", "xxaab") == diff2("axaxc", "xxaab", "xxaab")


In [60]:

def average(l):
    return sum(l) / len(l)

def score_all(word, corpus):
    def score_one(w1, w2):
        a, b = score_inline(w1, w2)
        return a*WELL_PLACED_BONUS + b
    return average([score_one(word, w) for w in corpus])

def score2_all(diff_func, w1, w2, corpus):
    return average([score2_generic(diff_func, w, w1, w2) for w in corpus])

def score_n_all(words, corpus):
    def score_n_1(base, words):
        greens, yellows = diff_n(base, words)
        green_count = sum([1 if a else 0 for a in greens])
        return green_count*WELL_PLACED_BONUS + len(yellows)
    return average([score_n_1(w, words) for w in corpus])

In [ ]:
print(score_n_all(['SOARE', 'CLINT', 'PUDGY'], core))
print(score_n_all(['TRAIN', 'LOUSE', 'CHIMP'], core))
print(score_n_all(['SLATE', 'IRONY', 'CHUMP'], core))
print(score_n_all(['STAIR', 'CLONE', 'PUDGY'], core))

In [65]:
# Compute and cache scores

cached_scores = {}
first_two = ""
for w in extended:
    cached_scores[w] = score_all(w, core)


In [ ]:
# first attempt to find a starting pair of words

def find_pairs(corpus):
    result = []
    target_list = most_common(core, 10)
    target_set = set(target_list)
    corpus_set = set(corpus)
    first_two = ""
    tested = 0
    for word in corpus:
        if False and word[:2] != first_two:
            first_two = word[:2]
            print(f"{first_two} {tested} {len(result)}")
            tested = 0
        letter_set = set(word)
        if len(letter_set) != 5:
            continue
        if not letter_set.issubset(target_set):
            continue
        tested += 1
        remaining = target_set - letter_set
        for perm in permutations(remaining):
            candidate = ''.join(perm)
            if candidate in corpus_set:
                score1 = cached_scores[word]
                score2 = cached_scores[candidate]
                result.append((score1 + score2, score1, word, candidate))
        
    return sorted(result)


result_core = find_pairs(core)
result_core.reverse()
result_extended = find_pairs(extended)
result_extended.reverse()

In [ ]:
# print results

def format(word):
    if word in core:
        return f"**{word}**"
    else:
        return word

for result in (result_core, result_extended):
    print(len(result))
    print('\n'.join([f"{format(r[2])}, {format(r[3])} ({r[0]:.3f})" for r in result[0:20:2]]))

    scores = pd.DataFrame(result, columns=['score', 'score1', 'word1', 'word2'])
    scores.hist(column='score', bins=10)


In [ ]:
# stats on word scores

word_scores = pd.DataFrame(cached_scores.items())
word_scores.columns = ["word", "score"]
core_set = set(core)
word_scores["core"] = word_scores.apply(lambda r: r["word"] in core_set, axis=1)
word_scores["sort_key"] = word_scores.apply(lambda r: (r["score"],r["word"]), axis=1)
print(word_scores)
word_scores.hist(column="score", bins=50)
m = word_scores['score'].median()
print(word_scores.min())
print(word_scores.max())
print(word_scores.mean())
med = [w for (w,s) in cached_scores.items() if abs(s - m) < 10**-8]
print([(x, cached_scores[x]) for x in med])



In [ ]:
print(word_scores[word_scores.core == True].min())

In [ ]:
# find a 3rd word for the first method

third_word_candidates = most_common(core, 26)[10:]
print(third_word_candidates)

third_words = []
for perm in permutations(third_word_candidates, 5):
    candidate = ''.join(perm)
    if candidate in cached_scores:
        # third_words.append([candidate, cached_scores[candidate], candidate in core])
        third_words.append((candidate, cached_scores[candidate], candidate in core))
third_words.sort(key=lambda r: r[1], reverse=True)

def format(w,c):
    if c:
        return f"**{w}**"
    else:
        return w
for (w, s, c) in third_words:
    print(f"|{format(w, c)} | {s:.3f} |")    

In [67]:
cached_scores["SOARE"] + cached_scores["CLINT"] # + cached_scores["PUDGY"]

4.1818574514038875

In [69]:

# Second attempt to find starting pair

extended_by_score = [x[0] for x in sorted(cached_scores.items(), key=lambda x: x[1], reverse=True)]
core_set = set(core)
core_by_score = [x for x in extended_by_score if x in core_set]

def find_best_pairs(corpus, threshold=4):
    best = 0
    h= []
    tested, skipped = 0, 0
    next_thresh = 1000
    n = len(corpus)
    for i in range(n):
        w1 = corpus[i]
        score1 = cached_scores[w1]
        if score1*2 < threshold:
            print(f'Stopping at index {i} ({w1}); score = {score1}')
            break
        for j in range(i+1, n):
            w2 = corpus[j]
            score2 = cached_scores[w2]
            if score1 + score2 < threshold:
                skipped += n - j
                break
            if tested + skipped >= next_thresh:
                print(f'(tested {tested}, skipped {skipped})')
                next_thresh = ((tested + skipped) / 1000 + 1) * 1000
            tested += 1
            score = score2_all(diff2_inline, w1, w2, core)
            h.append((score, w1, w2))
            if score > best:
                print(f'New best: {w1} {w2} {score} (tested {tested}, skipped {skipped})')
                best = score
        
    print(f"Tested: {tested}, skipped: {skipped}, total {skipped + tested}")
    return sorted(h, reverse=True)

In [ ]:
r1 = find_best_pairs(core_by_score)

# Took 6m24s with cache

In [ ]:
r2 = find_best_pairs(extended_by_score)

# Took 198m 44s without cache

In [ ]:
# Is there a  way to compute the score for a pair from indicidual scores ?

def score_diff(word1, word2):
    wp, np = score(word1, word2)
    return wp*WELL_PLACED_BONUS + np

    
correlation_analysis = pd.DataFrame(r1, columns=['score', 'word1', 'word2'])
correlation_analysis['score1'] = correlation_analysis.apply(lambda r: cached_scores[r['word1']], axis=1)
correlation_analysis['score2'] = correlation_analysis.apply(lambda r: cached_scores[r['word2']], axis=1)
correlation_analysis['total'] = correlation_analysis.apply(lambda r: r['score1'] + r['score2'], axis=1)
correlation_analysis['delta'] = correlation_analysis.apply(lambda r: r['total'] - r['score'], axis=1)
correlation_analysis['diff'] = correlation_analysis.apply(lambda r: score_diff(r['word1'], r['word2']), axis=1)
print(correlation_analysis)

In [ ]:
plot = px.scatter(correlation_analysis, x="delta", y="diff")
plot.show()

In [8]:
extended_pairs = json.load(open('extended_pairs.json'))
core_pairs = json.load(open('core_pairs.json'))

In [ ]:
# print results for second method

def format(word):
    if word in core:
        return f"**{word}**"
    else:
        return word

for result in (core_pairs, extended_pairs):
    print(len(result))
    print('\n'.join([f"| {format(r[1])} | {format(r[2])} | {r[0]:.3f}|" for r in result[0:10]]))

In [ ]:
# Find 3rd word for second method (Note: outdated, see truplets.py)

def search(pairs, corpus, threshold=5):
    best = 0
    results = []
    for p in pairs:
        print(f"***** {p[1]} {p[2]} *****")
        pair_score = p[0]
        for third in corpus:
            third_score = cached_scores[third]
            if pair_score + third_score < threshold:
                break
            triplet = (p[1], p[2], third)
            score = score_n_all(triplet, core)
            results.append((score, *triplet))
            if score > best:
                print(f'New best: {triplet} {score}')
                best = score
    return sorted(results, reverse=True)

r1 = search(core_pairs[:100], core_by_score)

In [6]:
# the triplets files are generated by triplets.py

extended_triplets = json.load(open('extended_triplets.json'))
core_triplets = json.load(open('core_triplets.json'))


In [ ]:
def lookup(triplet, pairs):
    for i, p in enumerate(pairs):
        if p[1] == triplet[1] and p[2] == triplet[2]:
            return i

for r in extended_triplets[:10]:
    i = lookup(r, extended_pairs)
    print(f"| {format(r[1])} | {format(r[2])} | {format(r[3])} | {r[0]:.3f}| {i}| ")

for r in core_triplets[:20]:
    i = lookup(r, core_pairs)
    print(f"| {r[1]} | {r[2]} | {r[3]} | {r[0]:.3f}| {i}| ")

#print(extended_triplets[:20])
#print("*****")
# print(core_triplets[:20])

In [64]:
# print out the scores for some triplets

p_e = extended_pairs[0][1:]
c_e = core_pairs[0][1:]

for i, t in enumerate(extended_triplets):
    if t[1] == p_e[0] and t[2] == p_e[1]:
        print(i, t)
        break

for i, t in enumerate(core_triplets):
    if t[1] == c_e[0] and t[2] == c_e[1]:
        print(i, t)
        break

# print(score_n_all(["TRAIN", "LOUSE", "CHIMP"], core))
def score_n_with_details(words, corpus):
    def score_n_1(base, words):
        greens, yellows = diff_n(base, words)
        green_count = sum([1 if a else 0 for a in greens])
        return (green_count, len(yellows))
    greens, yellows = 0, 0
    for w in corpus:
        g, y = score_n_1(w, words)
        greens += g
        yellows += y
    return (greens/len(corpus), yellows/len(corpus))

g1, y1 = score_n_with_details(["SLATE", "IRONY"], core)
g2, y2 = score_n_with_details(["TRAIN", "LOUSE"], core)
print(f"{g1:.3f} {y1:.3f} {g2:.3f} {y2:.3f}")
print(32*(g1 - g2), 32*(y1 - y2), 32*(g1 - g2)*2 + 32*(y1 - y2))
g3, y3 = score_n_with_details(["SLATE"], core)
g4, y4 = score_n_with_details(["TRAIN"], core)
print(32*(g3 - g4), 32*(y3 - y4), 32*(g3 - g4)*2 + 32*(y3 - y4))


16 [5.365442764578834, 'CRATE', 'SOILY', 'BUNDH']
51 [5.230669546436285, 'SLATE', 'IRONY', 'CHUMP']
1.092 1.941 0.924 2.127
5.390928725701944 -5.943844492440604 4.838012958963283
5.888552915766738 -1.9075593952483842 9.869546436285091


In [30]:
# check that on average, 2 words have a score of ~1.5

import random

def score_one(w1, w2):
    greens, yellows = diff_inline(w1, w2)
    a = sum([1 if a else 0 for a in greens])
    return a*2 + len(yellows)

n = 10**5
tot_g, tot_y = 0, 0
for _ in range(n):
    w1 = random.choice(core)
    w2 = random.choice(extended)
    g, y = diff_inline(w1, w2)
    tot_g += sum([1 if a else 0 for a in g])
    tot_y += len(y)
print(tot_g/n, tot_y/n, (tot_g*2 + tot_y)/n)

0.34999 0.84461 1.54459


In [ ]:
# search for triplets with the same score
scores = defaultdict(set)
for s in (extended_triplets, core_triplets):
    for score, *triplet in s:
        scores[score].add(tuple(sorted(triplet)))

sets = []
for triplets in scores.values():
    if len(triplets) > 1:
        sets.append(len(triplets))
print(sorted(sets, reverse=True))